# Import lots of stuff

In [2]:
import sys
sys.path.append('../')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import math
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zss import simple_distance
from nltk import Tree
#from utils import nlp_utils as utils
#from nltk import bigrams
from collections import Counter
#from pycorenlp import StanfordCoreNLP
%matplotlib inline
pd.set_option('display.max_rows', 500)

In [5]:
import spacy
from spacy.lang.en import English
nlp = spacy.load('en')

# Preprocess data for analyses

## Import annotated file

In [6]:
d_raw = pd.read_csv('../../data/tangramsSequential.csv')
d = d_raw.copy()

# Result 1: Generate file for POS analysis

## Get part of speech

## Compute edit distances on successive rounds

In [7]:
from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
archive = load_archive(
            "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz"
        )
predictor = Predictor.from_archive(archive, 'constituency-parser')

<class 'allennlp.modules.text_field_embedders.text_field_embedder.TextFieldEmbedder'>
<class 'allennlp.modules.span_extractors.span_extractor.SpanExtractor'>
<class 'allennlp.modules.seq2seq_encoders.seq2seq_encoder.Seq2SeqEncoder'>
<class 'allennlp.modules.feedforward.FeedForward'>
<class 'allennlp.modules.token_embedders.embedding.Embedding'>
<class 'allennlp.nn.initializers.InitializerApplicator'>


In [30]:
def label_distance (label1, label2) :
    if label1 == label2 :
        return 0
    else :
        return 1

def get_root(doc) :
    asdf = dict(('label' if key == 'nodeType' else key, value) for (key, value) in doc.items())
    return asdf

def get_children(subtree) :
    if 'children' in subtree.keys() :
        return [dict(('label' if key == 'nodeType' else key, value) for (key, value) in d.items()) for d in subtree['children']]
    else :
        return []

def get_label(node) :
    return node['label']

def edit_distance(tree1, tree2, return_operations = False) :
    return simple_distance(get_root(tree1), get_root(tree2), 
                           get_children, get_label, label_distance, return_operations=return_operations)

def example() :
    s1 = predictor.predict_json({"sentence": "you are a dog with a big bone"})
    s2 = predictor.predict_json({"sentence": "I am a cat"})
    print('tree1:', Tree.fromstring(s1['trees']))
    print('tree2:', Tree.fromstring(s2['trees']))
    print('operations:', edit_distance(s1, s2, return_operations=True)[1])
    print('cost:', edit_distance(s1, s2, return_operations=True)[0])

In [38]:
mini_data = d.query('role == "director" and gameid == "0057-414228f8-c268-40d6-9349-b35df4f080d9"')
mini_data['tree_parse'] = 

/Users/rxdh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
from IPython.display import clear_output
gameids = np.unique(d['gameid'])
tangramids = np.unique(d['intendedName'])

game_d = d.query('role == "director"')[['intendedName', 'occurrenceNum', 'contents']]
parses = []
for i, s in enumerate(game_d['contents']) :
    clear_output(wait=True)
    print(i, '/', len(game_d['contents']))
    parses.append(predictor.predict_json({'sentence' : s})['hierplane_tree']['root'])

156 / 8759


KeyboardInterrupt: 

In [57]:
transitions = np.zeros([12,5])
for i, gameid in enumerate(gameids) :
    print(i, '/', len(gameids))
    for j, intendedName in enumerate(tangramids) :
        for k, init_occurrenceNum in enumerate(range(1,6)) :
            dist = []
            sub1 = game_d.query('intendedName == "{0}" and occurrenceNum == {1}'.format(intendedName, init_occurrenceNum))['tree_parse']
            sub2 = game_d.query('intendedName == "{0}" and occurrenceNum == {1}'.format(
                intendedName, init_occurrenceNum+1
            ))['tree_parse']
            for tree1 in sub1 :
                for tree2 in sub2 :
                    dist.append(edit_distance(tree1, tree2))
            transitions[i, j, k] = np.max(dist)

0 / 92


NameError: name 'tangramsids' is not defined

In [48]:
# docs = nlp.pipe([unicode(s, 'utf-8') for s in d['contents']], batch_size=50)
# doc1 = docs.next()
# doc2 = docs.next()
# print(doc1.print_tree())
# print(edit_distance(doc1, doc2))
def compute_edit_distances(grp) :
    for i in 
    grp.groupby(['occurrenceNum', 'intendedName'])

    print([k for k in grp.contents])

In [49]:
# A lemma is a (word, pos) tag pair.
(d
 .query('role == "director"')
 .groupby(['gameid', 'intendedName'])
 .pipe(compute_edit_distances))

[(('0057-414228f8-c268-40d6-9349-b35df4f080d9', 1, 'A'), 2178    This looks like one of the Spy Vs Spy guys loo...
Name: contents, dtype: object), (('0057-414228f8-c268-40d6-9349-b35df4f080d9', 1, 'B'), 2166                                                   Hi
2168    Looking for a diamond at the top, triangle poi...
2169             close to a person praying on their knees
2171    There's a triangle on the right also. Just bel...
2172          Could also be seen as a mouse in silhouette
2174    Like a mountain - we've got 72 of these to get...
Name: contents, dtype: object), (('0057-414228f8-c268-40d6-9349-b35df4f080d9', 1, 'C'), 2183    preacher with hands up in ai
2184                             air
Name: contents, dtype: object), (('0057-414228f8-c268-40d6-9349-b35df4f080d9', 1, 'D'), 2185    Graduate giving speech at lectern, in profile
2187              The lectern is pointing to the left
Name: contents, dtype: object), (('0057-414228f8-c268-40d6-9349-b35df4f080d9', 1, 'E'), 218

In [ ]:
d['tokens'] = [[element[0] for element in l] for l in d['lemmas']]
d['pos'] = [[element[1] for element in l] for l in d['lemmas']]

In [ ]:
d['numWords'] = [pd.value_counts(words).sum() for words in d['tokens']]

In [ ]:
d['parse'] = [[w for w in utils.stanford_parsetree(text)] for text in d['contents']]

In [ ]:
Tree.fromstring(d['parse'][10][0])

In [ ]:
d['num_sbar'] = [utils.sbar_count(parse[0]) for parse in d['parse']]
d['num_pp'] = [utils.pp_count(parse[0]) for parse in d['parse']]
d['num_cc'] = [utils.cc_count(parse[0]) for parse in d['parse']]

## Get counts for each POS label

In [ ]:
d['nouns'] = [sum([1 if utils.is_noun(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['prepositions'] = [sum([1 if utils.is_prep(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['verbs'] = [sum([1 if utils.is_verb(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['determiners'] = [sum([1 if utils.is_det(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['pronouns'] = [sum([1 if utils.is_pronoun(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['adjectives'] = [sum([1 if utils.is_adjective(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['adverbs'] = [sum([1 if utils.is_adverb(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['numbers'] = [sum([1 if utils.is_num(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['others'] = [sum([1 if utils.is_other(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]

In [ ]:
d.columns

## Export to csv for plotting in R

In [ ]:
(d.drop(["lemmas", "tokens"], 1)
 .to_csv("sequential_posTagged.csv", index = False))

# Result 2: Calculate indicator words for tangrams/rounds

## First, get list of words in first round

In [ ]:
# Filter down to first round
d_round1 = d[d['occurrenceNum'] == 1]

# Pull out all tokens and collapse into count dict
tokenDict = Counter([item for sublist in d_round1['tokens'].tolist()
                     for item in sublist])
# Pull out all words that occur more than once
wordList = [word for (word,count) in tokenDict.items() if count > 1 and not word.isdigit()]
print(wordList[0:10])
print(len(wordList))

# Get POS map; will be longer because it doesn't require count > 1, but it doesn't matter
POSdict = {word: POS for lemma in d_round1['lemmas'] for (word, POS) in lemma}
print(len(POSdict.keys()))

## Get list of all game ids

In [ ]:
gameidList = pd.unique(d.gameid.ravel()).tolist()
print(gameidList[0:5])
print(len(gameidList))

## Get list of all tangram names

In [ ]:
tangramList = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
print(tangramList)

## Helper functions to select words & counts

In [ ]:
def getWordCounts(df, gameid, occurrenceNum, tangram = None) :
    roundCond = 'occurrenceNum == ' + occurrenceNum
    gameidCond = 'gameid == "' + gameid + '"'
    if(tangram is not None) :
        tangramCond = 'intendedObj == "' + tangram + '"'
        cond = " and ".join((roundCond, gameidCond, tangramCond))
    else :
        cond = " and ".join((roundCond, gameidCond))
    relevantRow = df.query(cond)
    return Counter([item for sublist in relevantRow['tokens'].tolist() 
                    for item in sublist])

#creates mini dataframe that grabs the words used in round n for a given tangram and gameid
def selectTangramRoundWords(df, tangram, roundNum, gameid):
    wordCounts = getWordCounts(df, gameid, roundNum, tangram)
    return list(wordCounts.keys())

## Helper function to compute PMIs

Note that merging is really costly -- if we need to speed it up, this might be the first target. Can also vectorize the log operation.

In [ ]:
#returns a table with the all words above 0 PMI and their counts for a given tangram
#calculate the probability for words given tangram A ------ p(x|y)
def makeMyPMI(df, tangram, roundNum, gameid, totals):

    # count words w/in tangram
    tangramCounts = getWordCounts(df, gameid, roundNum, tangram)

    #total number of words 
    tangramNumWords = (1 if sum(tangramCounts.values()) == 0 
                       else sum(tangramCounts.values()))

    #dataframe to compare 
    indicatorDF = pd.merge(pd.DataFrame(list(tangramCounts.items()), columns=['word', 'count']),
                           pd.DataFrame(list(totals["counts"].items()), columns=['word', 'totalCount']),
                           on='word', how = 'inner')

    #calculate PMI without log first. Having trouble with float issues. 
    indicatorDF['roughPMI'] = ((indicatorDF['count']/tangramNumWords)
                                / (indicatorDF['totalCount']/totals["numWords"]))
    indicatorDF['logPMI'] = [math.log10(num) for num in indicatorDF['roughPMI']]
    
    #remove column rough PMI
    indicatorDF = indicatorDF.drop('roughPMI', 1)
    
    return indicatorDF

## Write out PMIs & matching rates for all words

Note that we do a sloppy optimization by only computing total counts once and only when necessary 

In [ ]:
def memoize(d, gameid, counts) : 
    if "counts" not in counts : 
        counts["counts"] = getWordCounts(d, gameid, "1")
        counts["numWords"] = float(sum(counts["counts"].values()))
        return counts
    else :
        return counts

with open('sequential_matchAndPMI.csv', 'a', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['word', 'POS', 'match', 'pmi', 'total'])
    for word in wordList :
        print(word + ":" + POSdict[word])
        pmi = 0
        match = 0
        total = 0
        for gameid in gameidList:  
            memoizedCounts = {}
            for tangram in tangramList:
                memoizedCounts = memoize(d, gameid, memoizedCounts)
                round1WordList = selectTangramRoundWords(d, tangram, "1", gameid)
                total = total + 1 if word in round1WordList else total
                if word in round1WordList :
                    PMI_df = makeMyPMI(d, tangram, "1", gameid, memoizedCounts)
                    pmi = pmi + PMI_df[PMI_df['word'] == word]['logPMI'].tolist()[0]
                    round6WordList = selectTangramRoundWords(d, tangram, "6", gameid)
                    match = (match + 1 if (word in round1WordList and word in round6WordList)
                             else match)
        writer.writerow([word, POSdict[word], float(match) / float(total), pmi/total, total])


Also write out raw word counts on each round (so we can see what most likely words to be dropped are).

In [ ]:
for row in d :
    print row

## Compute unigrams and bigrams

In [ ]:
d['bigrams'] = [list(bigrams(l)) for l in d['tokens']]
bigramDict = Counter([item for sublist in d['bigrams'].tolist()
                     for item in sublist])
bigramList = [bigram for (bigram,count) in bigramDict.items() if count > 5]

In [ ]:
def getBigramCounts(df, gameid, roundNum, tangram = None) :
    roundCond = 'roundNum == ' + roundNum
    gameidCond = 'gameid == "' + gameid + '"'
    if(tangram is not None) :
        tangramCond = 'tangramRef == "' + tangram + '"'
        cond = " and ".join((roundCond, gameidCond, tangramCond))
    else :
        cond = " and ".join((roundCond, gameidCond))
    relevantRow = df.query(cond)
    return Counter([item for sublist in relevantRow['bigrams'].tolist() 
                    for item in sublist])

with open('bigramCounts.csv', 'a') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['gameid', 'roundNum', 'word', 'count'])
    for gameid in gameidList:  
        for roundNum in ['1', '2', '3', '4', '5', '6'] :
            counts = getBigramCounts(d, gameid, roundNum)
            for bigram in bigramList :
                writer.writerow([gameid, roundNum, ' '.join(bigram), counts[bigram]])

In [ ]:
## bootstrap analysis (might want to move to R)

TODO: exclude numbers earlier in the pipeline, 

TODO: don't average over matches and pmis

In [ ]:
numSamples = 1000

#grab words with highestPMI for a given tangram/gameid
def highestPMIWords(d, tangram, roundNum, gameid):
    allTangramCounts = {}
    allTangramCounts['counts'] = getWordCounts(d, gameid, "1")
    allTangramCounts['numWords'] = float(sum(allTangramCounts["counts"].values()))

    PMIdf = (makeMyPMI(d, tangram, roundNum, gameid, allTangramCounts))

    # Remove numbers
    PMIdf['POS'] = [POSdict[word] for word in PMIdf['word']]
    PMIdf = PMIdf.query('POS != "CD"'.format())

    #if PMIdf has words, pull out max values, it is empty return it as is
    if len(PMIdf.index) > 0:
        PMI_values = PMIdf.logPMI.unique()
        maxPMI = PMI_values.max()
        PMIdf = PMIdf.loc[PMIdf['logPMI'] == maxPMI]
        PMIdfword = PMIdf['word']
        return PMIdfword.tolist()
    else: 
        return PMIdf

with open('PMIbootstrap.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['sampleNum', 'tangram', 'gameid', 'numCandidates', 'match', 'highest'])
    for gameid in gameidList :
        for tangram in tangramList :
            round1Words = selectTangramRoundWords(d, tangram, "1", gameid)
            if len(round1Words) > 0:
                # First, write highest PMI match
                highPMIWords = highestPMIWords(d, tangram, "1", gameid)
                round6Words = selectTangramRoundWords(d, tangram, "6", gameid)
                match = np.mean([1 if word in round6Words else 0 for word in highPMIWords ])
                writer.writerow([0, tangram, gameid, len(highPMIWords), match, "highest"])

                # Next, take a bunch of null samples
                for i in range(numSamples) :
                    randomWord = np.random.choice(round1Words)
                    match = np.mean([1 if randomWord in round6Words else 0])
                    writer.writerow([i + 1, tangram, gameid, 1, match, "null"])